In [1]:
import numpy as np 
import pandas as pd 
from os import path
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, Lambda
from keras.models import Model
from keras.utils import np_utils,Sequence
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler,EarlyStopping,ModelCheckpoint

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

%matplotlib inline

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333) 
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3517582661897484957
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3169819033
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3604979518748665431
physical_device_desc: "device: 0, name: Quadro P1000, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
images_dir = 'original_dataset/'
test = 'test_WyRytb0.csv'
train_dir = 'train/'
test_dir = 'test/'
labels = 'train.csv'
sample = 'sample_submission.csv'
IMG_SIZE = 150
batch_size = 4
train_split = 0.85

In [4]:
# All the labels for the training data
y_train = pd.read_csv(labels)
y_train.head()

,image_name,label
0,0.jpg,0
1,1.jpg,4
2,2.jpg,5
3,4.jpg,0
4,7.jpg,4


In [5]:
# This will help in preparing the images for test data
y_test = pd.read_csv(test)
y_test.head()

,image_name
0,3.jpg
1,5.jpg
2,6.jpg
3,11.jpg
4,14.jpg


In [6]:
def prepareImages(data, m):
    print("Preparing images")
    X_train = np.zeros((m, 150, 150, 3))
    count = 0
    
    for fig in data['image_name']:
        #load images into images of size 150*150(original size)
        #src = cv2.imread("train/"+fig,)
        #img = cv2.cvtColor(src,cv2.COLOR_BGR2RGB)
        img = image.load_img("original_dataset/"+fig, target_size=(150, 150, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        X_train[count] = x
        count += 1
        if(count%1000==0):
            print('images done :',count)
    np.save('traindata.npy',X_train)
    return X_train

In [12]:
#If you already have npy file load that
#X_train = prepareImages(y_train,y_train.shape[0])
X_train = np.load('traindata.npy')


In [13]:
#X_train = X_train.reshape(X_train.shape[0],3,150,150)
#X_train = X_train.astype('float32')
X_train /=255

In [14]:
# fix random seed for reproducibility
random_seed = 7
np.random.seed(random_seed)

In [15]:
# one hot encode outputs'
y_train_label = np_utils.to_categorical(y_train['label'])
num_classes = y_train_label.shape[1]
num_classes

6

In [16]:
# Split in Training set and Validation set
x_train2, x_val, y_train2, y_val = train_test_split(X_train, y_train_label, test_size = 0.15, random_state=random_seed)
print (x_train2.shape, y_train2.shape, x_val.shape, y_val.shape)

(14478, 150, 150, 3) (14478, 6) (2556, 150, 150, 3) (2556, 6)


In [17]:
def ConvBlock(layers, model, num_filters):
    """
    Create a layered Conv/Pooling block
    """
    for i in range(layers):
        model.add(Conv2D(num_filters, (3, 3), activation='relu', padding='same')) # 3x3 filter size 
        
    model.add(MaxPooling2D((2,2), strides=(2,2), data_format='channels_last'))

def FCBlock(model, size=1024):
    """
    Fully connected block with ReLU and dropout
    """
    model.add(Dense(size, activation='relu'))
    model.add(Dropout(0.4))
    
def my_VGG16(input_shape):
    """
    Implement VGG16 architecture
    """
    model = Sequential()
    model.add(Lambda(lambda x : x, input_shape=input_shape))
    
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(2, model, 256)
    ConvBlock(2, model, 512)
    #ConvBlock(3, model, 512)

    model.add(Flatten())
    FCBlock(model)
    #FCBlock(model)

    model.add(Dense(6, activation = 'sigmoid'))
    return model

In [18]:
im_shape = (IMG_SIZE,IMG_SIZE,3)
# Then create the corresponding model 
my_model = my_VGG16(im_shape)
my_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 75, 75, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 128)       0         
__________

In [19]:
# With data augmentation to prevent overfitting
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        )  # randomly flip simages


datagen.fit(X_train)

    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
validation_split=0.0)

In [20]:
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
    
# descriptive weight file naming
checkpointer = ModelCheckpoint(filepath=('vgg16_2.h5'), 
                               verbose=1, save_best_only=True)

callbacks = [annealer,early_stopping,checkpointer]

In [21]:
batch_size = 8
steps_per_epoch = x_train2.shape[0]//batch_size
validation_steps = x_val.shape[0]//batch_size

In [22]:
hist = my_model.fit_generator(datagen.flow(x_train2,y_train2,batch_size=batch_size),
    validation_data = (x_val,y_val),            
    validation_steps = validation_steps,
    epochs=50,
    verbose=1,
    steps_per_epoch = steps_per_epoch,
    callbacks=callbacks
)

Epoch 1/50
1809/1809 [==============================] - 424s 234ms/step - loss: 0.4577 - acc: 0.8325 - val_loss: 0.4511 - val_acc: 0.8333

Epoch 00001: val_loss improved from inf to 0.45112, saving model to vgg16_2.h5
Epoch 2/50
  59/1809 [..............................] - ETA: 7:43 - loss: 0.4518 - acc: 0.8333

KeyboardInterrupt: 

In [18]:
def prepareTest(data, m):
    print("Preparing images")
    X_train = np.zeros((m, 150, 150, 3))
    count = 0
    
    for fig in data['image_name']:
        #load images into images of size 150*150(original size)
        #src = cv2.imread("train/"+fig,)
        #img = cv2.cvtColor(src,cv2.COLOR_BGR2RGB)
        img = image.load_img("original_dataset/"+fig, target_size=(150, 150, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        X_train[count] = x
        count += 1
        if(count%1000==0):
            print('images done :',count)
    np.save('test.npy',X_train)
    return X_train

In [19]:
x_test = prepareTest(y_test,y_test.shape[0])
x_test /= 255

Preparing images
images done : 1000
images done : 2000
images done : 3000
images done : 4000
images done : 5000
images done : 6000
images done : 7000


In [20]:
x_test.shape

(7301, 150, 150, 3)

In [22]:
result = np.zeros((x_test.shape[0],6))
result = my_model.predict(x_test,batch_size=8)
result = np.argmax(result,axis=1)
results = pd.Series(result,name='label')

In [35]:
y_test['label'] = results
y_test.to_csv('vgg_16.csv',index=False)